In [ ]:
#install.packages('ppcor')
library(ppcor)
library(lmPerm)
library(psych)
library(car)
library(ggplot2)
#install.packages('lme4')
library(lme4)
library(mediation)
#install.packages('caret')
#library(caret)
#install.packages('gbm')
library(gbm)
#install.packages('party')
library(party)
#install.packages('Metrics')
library(Metrics)
#install.packages("wesanderson")
library(wesanderson)
#install.packages('gtools')
library(gtools)
#install.packages('interactions')
library(interactions)
library(dplyr)
library(ukbtools)
library(mediation)
library(lavaan)
#install.packages('Deducer')
#library(Deducer)
#install.packages('gridExtra')
library(gridExtra)
#install.packages('reshape2')
library(reshape2)
#install.packages('ggsignif')
library(ggsignif)
#install.packages('svglite')
library(svglite)
library(MatchIt)

In [ ]:
di=read.table('/dagher/dagher11/filip/PRSOB/data/brain_data.csv', header=TRUE, na.strings = "", sep=',', quote = "\"")


# Exclude participants who had a heart attack, angina or stroke - self-reported
exclusions_vd=c('Angina','Heart attack','Stroke')
di$vascular_heart_diagnoses <- NA
for (i in as.numeric(grep('vascularheart_problems_diagnosed_by_doctor', colnames(di)))) {
    for (d in 1:length(exclusions_vd)) {
        di$vascular_heart_diagnoses[grepl(exclusions_vd[d], di[[i]])] = 1
    }
}
             
exclusions_neuro=c('^G','^E0','^E10','^E2','^E3','^A8','^C70','^C71','^C72','^I6')
# G - nervous system; F - Mental and behavioural; E0 - thyroid; E11-14 - diabetes, E2-3 endocrine glands; 
# A8 - viral infections of the CNS; C70-72 malignant neoplasms of brain, meninges and spinal cord; I6 - cerebrovscular

di$neurological_disorder = NA
for (i in as.numeric(grep('icd10', colnames(di)))) {
    for (d in 1:length(exclusions_neuro)) {
        di$neurological_disorder[grepl(exclusions_neuro[d], di[[i]])] = 1
    }
}

di$excluded <- NA
di$excluded[di$vascular_heart_diagnoses ==1] <- 1
di$excluded[di$neurological_disorder ==1] <- 1

di$included<-car::recode(di$excluded, "1='excluded'; else='included'")
di=(di[!is.na(di$mean_thickness_of_caudalanteriorcingulate_left_hemisphere_27174.2.0),])
nrow(di)
#Select subjects if included ==1
di_excluded <- subset(di, included=="included")

ukbb_all=di_excluded
ukbb_all=(ukbb_all[!is.na(ukbb_all$mean_thickness_of_caudalanteriorcingulate_left_hemisphere_27174.2.0),])
nrow(ukbb_all)

ukbb_all$Hypertension=0
ukbb_all$Hypertension[ukbb_all$hypertension=='1' | ukbb_all$hypertension2=='1']=1
ukbb_all$Diabetes=0
ukbb_all$Diabetes[ukbb_all$diabetes=='1' | ukbb_all$diabetes2=='1']=1
ukbb_all$WHR = ukbb_all$waist_circumference_48.0.0/ukbb_all$hip_circumference_49.0.0
ukbb_all$agesq=ukbb_all$age_when_attended_assessment_centre_21003.2.0^2

#ukbb_all=(ukbb_all[!is.na(ukbb_all$mean_thickness_of_caudalanteriorcingulate_left_hemisphere_27174.2.0),])
nrow(ukbb_all)

In [ ]:
write.table(ukbb_all,'/dagher/dagher11/filip/OBAL/UKBB_OHBM_excluded.csv',
            row.names=F,col.names=T, quote=T)

In [ ]:
ukbb_all=read.table('/dagher/dagher11/filip/OBAL/UKBB_OHBM_excluded.csv',
            header=T, quote='"')

# New UKB dataframe with only one BMI timepoint

In [ ]:
rm(UKB)
UKB=data.frame(ukbb_all$eid, ukbb_all$body_mass_index_bmi_21001.2.0,
               ukbb_all$age_when_attended_assessment_centre_21003.2.0,
               ukbb_all$sex_31.0.0,
               ukbb_all$volume_of_accumbens.area_left_hemisphere_26564.2.0,
               ukbb_all$volume_of_accumbens.area_right_hemisphere_26595.2.0,
               ukbb_all$volume_of_estimatedtotalintracranial_whole_brain_26521.2.0,
               ukbb_all$volume_of_caudate_right_hemisphere_26590.2.0, 
               ukbb_all$volume_of_caudate_left_hemisphere_26559.2.0, 
               ukbb_all$volume_of_thalamus.proper_right_hemisphere_26589.2.0, 
               ukbb_all$volume_of_thalamus.proper_left_hemisphere_26558.2.0,
               ukbb_all$volume_of_pallidum_right_hemisphere_26592.2.0, 
               ukbb_all$volume_of_pallidum_left_hemisphere_26561.2.0, 
               ukbb_all$volume_of_putamen_right_hemisphere_26591.2.0, 
               ukbb_all$volume_of_putamen_left_hemisphere_26560.2.0,
               ukbb_all$body_mass_index_bmi_21001.0.0)

colnames(UKB)=c('ID','BMI','Age','Gender','LeftAcc','RightAcc','eTIV','CaudateR','CaudateL','ThalamusR',
                'ThalamusL','PallidumR','PallidumL','PutamenR','PutamenL','BMI1')

UKB=na.omit(UKB)

UKB$RAccSt=((UKB$RightAcc/UKB$eTIV)-mean(UKB$RightAcc/UKB$eTIV))/sd(UKB$RightAcc/UKB$eTIV)
UKB$LAccSt=((UKB$LeftAcc/UKB$eTIV)-mean(UKB$LeftAcc/UKB$eTIV))/sd(UKB$LeftAcc/UKB$eTIV)
UKB$RPutSt=((UKB$PutamenR/UKB$eTIV)-mean(UKB$PutamenR/UKB$eTIV))/sd(UKB$PutamenR/UKB$eTIV)
UKB$LPutSt=((UKB$PutamenL/UKB$eTIV)-mean(UKB$PutamenL/UKB$eTIV))/sd(UKB$PutamenL/UKB$eTIV)
UKB$RPalSt=((UKB$PallidumR/UKB$eTIV)-mean(UKB$PallidumR/UKB$eTIV))/sd(UKB$PallidumR/UKB$eTIV)
UKB$LPalSt=((UKB$PallidumL/UKB$eTIV)-mean(UKB$PallidumL/UKB$eTIV))/sd(UKB$PallidumL/UKB$eTIV)
UKB$RCauSt=((UKB$CaudateR/UKB$eTIV)-mean(UKB$CaudateR/UKB$eTIV))/sd(UKB$CaudateR/UKB$eTIV)
UKB$LCauSt=((UKB$CaudateL/UKB$eTIV)-mean(UKB$CaudateL/UKB$eTIV))/sd(UKB$CaudateL/UKB$eTIV)
UKB$RThaSt=((UKB$ThalamusR/UKB$eTIV)-mean(UKB$ThalamusR/UKB$eTIV))/sd(UKB$ThalamusR/UKB$eTIV)
UKB$LThaSt=((UKB$ThalamusL/UKB$eTIV)-mean(UKB$ThalamusL/UKB$eTIV))/sd(UKB$ThalamusL/UKB$eTIV)

variables_for_outliers=c(16:25)
#variables_for_outliers=c(2)

for (j in variables_for_outliers) {
    Q3=as.numeric(quantile(UKB[j],0.75, na.rm=TRUE))
    Q1=as.numeric(quantile(UKB[j],0.25, na.rm=TRUE))
    upper=Q3+(2.2*(Q3-Q1))
    lower=Q1-(2.2*(Q3-Q1))
    UKB[j][UKB[j]<lower]=NA
    UKB[j][UKB[j]>upper]=NA
}

UKB=na.omit(UKB)
nrow(UKB)

## Calculate moderations

In [ ]:
stats=matrix()
ps=matrix()

vars=c('RAccSt$', 'LAccSt$',  
       'RPutSt$', 'LPutSt$',  
       'RPalSt$', 'LPalSt$', 
       'RCauSt$', 'LCauSt$', 
       'RThaSt$', 'LThaSt$'
       )


for (i in 1:length(vars)) {

a=(summary(lm(UKB[[grep(vars[i],colnames(UKB))]] ~ 
      scale(UKB$BMI) *
      scale(UKB$Age) +
      as.factor(UKB$Gender))))
    
    print(a)
    
    ps[i]=a$coefficients[5,4]
    stats[i]=a$coefficients[5,3]

}


stats_adj=p.adjust(ps,method='BH')
stats_adj
stats

## Plot all age moderations

In [ ]:
UKB$BMIsl=(log(UKB$BMI)-mean(log(UKB$BMI)))/sd(log(UKB$BMI))
UKB$Ages=scale(UKB$Age)

regions=data.frame(UKB$RAccSt,UKB$LAccSt,
                   UKB$RPutSt,UKB$LPutSt,
                   UKB$RPalSt,UKB$LPalSt,
                   UKB$RCauSt,UKB$LCauSt,
                   UKB$RThaSt,UKB$LThaSt)

names=c('Right accumbens volume','Left accumbens volume',
        'Right putamen volume','Left putamen volume',
        'Right pallidum volume','Left pallidum volum',
        'Right caudate volume','Left caudate volume',
        'Right thalamus volume','Left thalamus volume')

tags1=c('c','a','c','a','c','a','c','a','c','a')
tags2=c('d','b','d','b','d','b','d','b','d','b')
        
filenames=c('Accumbens','Putamen','Pallidum','Caudate','Thalamus')
colours=c("BuPu","BuPu","YlGn","YlGn","RdPu","RdPu",
         "YlOrRd","YlOrRd",'Blues','Blues')
colours1=c('purple','purple','green3','green3',
           'magenta','magenta','goldenrod1','goldenrod1',
          'royalblue','royalblue')

significance=c('','p=0.017',
              '','',
              'p<0.003','p<0.001',
              '','',
              'p<0.023','')

slopes=list()
agedep=list()

for (i in 1:length(regions)) {
    
    a=as.numeric(regions[[i]])
    Reg=lm(a ~ BMIsl*Age + as.factor(Gender), data = UKB)
    
    ss=sim_slopes(Reg, pred = BMIsl, modx = Age, 
                     modx.values = c(45,50,55,60,65,70,75,80))
    
    slopes[[i]]=(plot(ss) + 
                 xlim(-0.2,0.2) + 
                 scale_color_manual(values=colours1[i]) + 
                 theme_classic() +
                 ylab(' ') + 
                 xlab(paste('Slope of BMI','*',names[i], sep=''))) + 
    labs(tag = tags1[i]) + 
    geom_text(x=0.15, y=8, label = significance[i], size=3.5, show.legend = FALSE, colour='black')
    

    agedep[[i]]=(interact_plot(Reg, pred = BMIsl, modx = Age, interval = TRUE, int.width = 0.8, 
                  color.class=colours[i], line.thickness=0.8) + 
                 theme_classic() + 
                 ylab(names[i]) + 
                 xlab('BMI z-scores')) + 
    labs(tag = tags2[i]) + 
    ylim(-1,1)

    
}

In [ ]:
p_index=1
for (p in 1:5) {        
    grid=grid.arrange(
      slopes[[p_index]], slopes[[p_index+1]], 
      agedep[[p_index]], agedep[[p_index+1]],
      widths = c(1,1),
      layout_matrix = rbind(c(2, 4),
                            c(1, 3)))
    
    p_index=p_index+2


    ggsave(
      paste('/dagher/dagher11/filip/OBAL/HBM_Figures/',filenames[p],'.tiff',sep=''),
      plot = grid,
      device = "tiff",
      scale = 1,
      width = 20,
      height = 20,
      units = c('cm'),
      dpi = 300)
    
    }

## Creating and matching obesity maintainers and developers

### Create grouping variables

In [ ]:
UKB$GroupBMI1=cut(UKB$BMI1,c(0,25,28.4,100))
UKB$GroupBMI3=cut(UKB$BMI,c(0,25,28.4,100))

### Divide into groups

In [ ]:
set.seed(03)
UKB_dfs=UKB %>%
  group_split(GroupBMI1, GroupBMI3)
UKB %>%
  group_by(GroupBMI1,GroupBMI3) %>%
summarize(n())

UKB_dfs=na.omit(UKB_dfs)

In [ ]:
Maintain=as.data.frame(UKB_dfs[[9]])
Maintain$Group='Maintainers'
Develop=as.data.frame(rbind(UKB_dfs[[3]]))
Develop$Group='Developers'

MainDev=rbind(Maintain,Develop)
MainDev$Sample <- as.logical(MainDev$Group == 'Developers')

### Match groups

In [ ]:
matched_groups <- matchit(Sample ~ BMI + Age + Gender, data = MainDev, method="nearest")
summary(matched_groups, standardize = TRUE)
#plot(matched_groups)

In [ ]:
df.match <- match.data(matched_groups)#[1:ncol(MainDev)]

### Plot BMIs

In [ ]:
library(tidyr)
gathered=gather(df.match, key='timepoint', value='BMI', c(BMI, BMI1), factor_key=FALSE)

sample_sum <- gathered %>%
  group_by(timepoint, Group) %>%
  summarize(mean = mean(BMI),
            sd   = sd(BMI),
            mean_p2sd = mean + 2 * sd,
            mean_m2sd = mean - 2 * sd) %>%
  ungroup()

sample_sum$timepoint[sample_sum$timepoint=="BMI"] <- 2
sample_sum$timepoint[sample_sum$timepoint=="BMI1"] <- 1
#sample_sum$timepoint[sample_sum$timepoint=="BMI2"] <- 2

#ggplot(data=gathered, aes(x=timepoint, y=BMI, group=Group, colour=Group)) + #geom_point(show.legend = FALSE) + 
#geom_line(show.legend=TRUE)
sample_sum$Group=factor(sample_sum$Group, ordered=TRUE)
sample_sum$timepoint=as.numeric(sample_sum$timepoint)

tiff('/dagher/dagher11/filip/OBAL/HBM_Figures/BMI_long.tiff', units="in", width=6, height=5, res=300)
ggplot(sample_sum, aes(x = (timepoint),  color = Group, fill = Group)) + 
  geom_ribbon(aes(ymin = mean_m2sd, ymax = mean_p2sd), alpha = 0.1) +
  geom_line(aes(y= mean)) +  theme_minimal() + labs(x = "Timepoint") +
scale_fill_manual(values=c("#530077", "#FF21FF")) + scale_color_manual(values=c("#530077", "#FF21FF")) +
ylab(bquote('BMI ['~'kg /'~~m^2~']')) + scale_x_continuous(breaks=c(1,2)) 
dev.off()




p <- ggplot(sample_sum %>%
              gather(stat, BMI, mean, mean_p2sd:mean_m2sd), 
            aes(x = timepoint, y = BMI)) + 
  geom_line(aes(color = Group, linetype = stat))+ facet_wrap(sample_sum$Group)


### Calculate stats 

In [ ]:
stats=matrix(14,1)
ps=matrix()

vars=c('LAccSt$', 'RThaSt$', 'RPalSt$', 'LPalSt$')


for (i in 1:length(vars)) {
    
    print(vars[i])
    
    a=(summary(lm(df.match[[grep(vars[i],colnames(df.match))]] ~ Group + Age + Gender, data = df.match)))
    
    stats[i]=a$coefficients[2,4]
    ps[i]=a$coefficients[2,3]
    print(a)
    
    }

vars
stats
ps

In [ ]:
Nacc_plot=select(df.match, ID, Group, LAccSt)
PalR_plot=select(df.match, ID, Group, RPalSt)
PalL_plot=select(df.match, ID, Group, LPalSt)
ThaR_plot=select(df.match, ID, Group, RThaSt)
Nacc_plot=melt(Nacc_plot,id.vars = c("ID",'Group'))
PalR_plot=melt(PalR_plot,id.vars = c("ID",'Group'))
PalL_plot=melt(PalL_plot,id.vars = c("ID",'Group'))
ThaR_plot=melt(ThaR_plot,id.vars = c("ID",'Group'))

In [713]:
df.match$GRP<-car::recode(df.match$Group, "'Maintainers'='2'; else='1'")


In [ ]:
graphs=list()


dframes=c('Nacc_plot', 'PalL_plot', 'PalR_plot','ThaR_plot')
titles=c('Left nucleus accumbens volume','Left pallidum volume',
         'Right pallidum volume','Right thalamus volume')
labels=c('','','0.026','')
colours=c('PRGn','RdPu','RdPu','Blues')
lines=c(0.85,1.85,1.15,2.15,0.85,1.85,0.85,1.85)
tags=c('a','b','d','c')
index=1
for (i in 1:length(dframes)) {

        graphs[[i]]=ggplot(eval(as.name(dframes[i])), aes(x=variable, y=value, fill=Group)) + 
            geom_violin(trim=FALSE,position=position_dodge(0.75), width=0.6) +
            geom_point(pch = 21, position = position_jitterdodge(dodge.width = 0.75, jitter.width=0.08), size=0.1) + 
            geom_boxplot(outlier.shape = NA, alpha=0.7, position=position_dodge(0.75), width=0.3) + 
            theme_minimal() +
            ylab(titles[i]) + 
            ylim(-4.5,4.5) +
            xlab('Software package') +
            scale_x_discrete(labels=c("Freesurfer", "FSL")) +
            theme(axis.title.x=element_blank()) +
            scale_fill_brewer(palette=colours[i]) +
            geom_text(x=1, y=4.4, label = labels[index], size=3) +
            geom_text(x=2, y=4.4, label = labels[index+1], size=3) +
            geom_segment(x = 0.85, xend = lines[index], y = 4, yend=4, size=0.05) +
            geom_segment(x = 1.85, xend = lines[index+1], y = 4, yend=4, size=0.05) +
            labs(tag = tags[i])
            
        index=index+2
    }


grid=grid.arrange(
  graphs[[1]], graphs[[2]], graphs[[3]], graphs[[4]],
  widths = c(1,1),
  layout_matrix = rbind(c(1, 2),
                        c(3, 4))
)

ggsave(
  '/dagher/dagher11/filip/OBAL/HBM_Figures/MainDev.tiff',
  plot = grid,
  device = "tiff",
  scale = 1,
  width = 20,
  height = 30,
  units = c('cm'),
  dpi = 300
)


In [503]:
structures=c('LAccSt$','LAccStFSL$','LPalSt$','LPalStFSL$','RPalSt$','RPalStFSL$','LPutSt$','LPutStFSL$',
             'RPutSt$','RPutStFSL$','LCauSt$','LCauStFSL$','RCauSt$','RCauStFSL$')
titles=c('Left nucleus accumbens volume - Freesurfer','Left nucleus accumbens volume - FSL',
         'Left pallidum volume - Freesurfer','Left pallidum volume - FSL','Right pallidum volume - Freesurfer',
         'Right pallidum volume - FSL','Left putamen volume - Freesurfer','Left putamen volume - FSL',
         'Right putamen volume - Freesurfer','Right putamen volume - FSL','Left caudate volume - Freesurfer',
         'Left caudate volume - FSL','Right caudate volume - Freesurfer','Right caudate volume - FSL')


In [504]:
for (i in 1:length(structures)) {
        
        graph=ggplot(df.match, aes(x=Group, y=df.match[[grep(structures[i],colnames(df.match))]])) + 
        geom_violin(trim=FALSE) +
        geom_boxplot(outlier.colour="red", outlier.shape=NA, outlier.size=4) + 
        geom_jitter(shape=16, position=position_jitter(0.1)) + 
        theme_minimal() +
        ylab(titles[i]) + 
        ylim(-5,5)
        
        print(graph)
        
}